In [4]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os.path

In [5]:
base = pd.read_csv("PETR4.SA.csv")

In [122]:
base = base.dropna()
base_treinamento = base.iloc[:,1:7].values

normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

normalizador_previsao = MinMaxScaler(feature_range=(0,1))
normalizador_previsao.fit_transform(base_treinamento[:,0:1])



array([[0.0264613 ],
       [0.02132701],
       [0.02044234],
       ...,
       [0.38909953],
       [0.39146919],
       [0.38404423]])

In [125]:
previsores = []
classe = []
#Cria um série temporal de 7 meses da coluna Open
for i in range(7,base_treinamento_normalizada.shape[0]):
  previsores.append(base_treinamento_normalizada[i-7:i,0:6]) #Coloca de colunas de 0-5
  classe.append(base_treinamento_normalizada[i,0])
previsores,classe = np.array(previsores), np.array(classe) 
#previsores = np.reshape(previsores,(previsores.shape[0],previsores.shape[1],1)) #batch = (batchsize=quantidade registros, time steps, numero de atributos previsores)
#previsores.shape

In [151]:
def redeNeuralRecorrente(prev,cl):
    regressor = Sequential()
    regressor.add(LSTM(units=100 , return_sequences=True, input_shape = (prev.shape[1],6)))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.3))
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.3))
    regressor.add(Dense(units=1,activation='linear')) #retorna valores entre 0-1
    if os.path.isfile('pesosRedeNeuralRecorrente.h5'):
        regressor.load_weights('pesosRedeNeuralRecorrente.h5') 
    regressor.compile(optimizer = 'adam', loss ='mean_squared_error', metrics = ['mean_absolute_error'])
    eas = EarlyStopping(monitor='loss',min_delta=1e-10,patience=5,verbose=1)#Se passarem 10 épocas, e não houver uma mudança mínima de 2e-05, ele para.
    rlc = ReduceLROnPlateau(monitor = 'mean_absolute_error', factor = 0.07,patience=2,verbose=1)#Reduz learning rate em 0.1, se não houver mudanças na loss function
    mcp = ModelCheckpoint(filepath='pesosRedeNeuralRecorrente.h5',monitor='mean_absolute_error',save_best_only=True)
    regressor.fit(prev,cl,epochs = 100, batch_size = 32,callbacks=[eas,rlc,mcp])
    return regressor

In [154]:
kfold = KFold(n_splits=30)

for train, test in kfold.split(previsores, classe):
  regressor = redeNeuralRecorrente(previsores[train], classe[train])

Epoch 1/100
160/160 [==============================] - 10s 21ms/step - loss: 2.7966e-04 - mean_absolute_error: 0.0120
Epoch 2/100
160/160 [==============================] - 3s 20ms/step - loss: 2.8640e-04 - mean_absolute_error: 0.0121
Epoch 3/100
160/160 [==============================] - 3s 20ms/step - loss: 2.8679e-04 - mean_absolute_error: 0.0120
Epoch 4/100
160/160 [==============================] - 3s 21ms/step - loss: 2.9780e-04 - mean_absolute_error: 0.0122
Epoch 5/100
160/160 [==============================] - 3s 21ms/step - loss: 3.1121e-04 - mean_absolute_error: 0.0123

Epoch 00005: ReduceLROnPlateau reducing learning rate to 7.00000033248216e-05.
Epoch 6/100
160/160 [==============================] - 3s 20ms/step - loss: 3.0423e-04 - mean_absolute_error: 0.0115
Epoch 7/100
160/160 [==============================] - 3s 20ms/step - loss: 2.6950e-04 - mean_absolute_error: 0.0113
Epoch 8/100
160/160 [==============================] - 3s 21ms/step - loss: 2.6131e-04 - mean_absolu

In [157]:
x_teste = [5.875000e+00, 5.875000e+00, 5.875000e+00, 5.875000e+00,
        3.941121e+00, 3.538944e+10]
x_teste +=  [5.550000e+00, 5.550000e+00, 5.550000e+00, 5.550000e+00,
        3.723102e+00, 2.886144e+10]
x_teste += [5.494000e+00, 5.494000e+00, 5.494000e+00, 5.494000e+00,
        3.685536e+00, 4.303360e+10]
x_teste += [5.475000e+00, 5.475000e+00, 5.475000e+00, 5.475000e+00,
        3.672790e+00, 3.405568e+10]
x_teste += [5.500000e+00, 5.500000e+00, 5.500000e+00, 5.500000e+00,
        3.689559e+00, 2.091264e+10]
x_teste += [5.613000e+00, 5.613000e+00, 5.613000e+00, 5.613000e+00,
        3.765363e+00, 1.956352e+10]
x_teste += [5.475000e+00, 5.475000e+00, 5.475000e+00, 5.475000e+00,
        3.672790e+00, 2.398720e+10]     
x_teste = np.array(x_teste)
x_teste = x_teste.reshape(7,6)
x_teste = normalizador.fit_transform(x_teste)
x_teste = x_teste.reshape(1,7,6)
previsao = regressor.predict(x_teste)
previsao = normalizador_previsao.inverse_transform(previsao)
previsao[0][0]

7.739349